In [ ]:
# %% [markdown]
# # 2D Elastic Collision Physics - Mathematical Foundation
# 
# This notebook explains the physics and mathematics behind the 2D elastic collision simulator.
# We'll cover momentum conservation, energy conservation, and the mathematical derivations
# used in the collision detection and response algorithms.

# %% [markdown]
# ## 1. Fundamental Physics Principles
# 
# ### Conservation Laws
# 
# Elastic collisions are governed by two fundamental conservation laws:
# 
# 1. **Conservation of Momentum**: The total momentum of the system remains constant
# 2. **Conservation of Kinetic Energy**: The total kinetic energy remains constant (no energy loss)

# %%
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.animation as animation
from IPython.display import HTML
import math

# Setup plotting
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(10, 6))

# %% [markdown]
# ### Mathematical Definitions
# 
# For a ball with mass $m$ and velocity vector $\vec{v} = (v_x, v_y)$:
# 
# **Momentum**: $\vec{p} = m\vec{v} = m(v_x, v_y)$
# 
# **Kinetic Energy**: $KE = \frac{1}{2}m|\vec{v}|^2 = \frac{1}{2}m(v_x^2 + v_y^2)$

# %%
def calculate_momentum(mass, velocity):
    """Calculate momentum vector"""
    return mass * np.array(velocity)

def calculate_kinetic_energy(mass, velocity):
    """Calculate kinetic energy"""
    return 0.5 * mass * np.sum(np.array(velocity)**2)

# Example calculation
mass1, velocity1 = 5.0, [10, 5]
mass2, velocity2 = 3.0, [-8, 12]

p1 = calculate_momentum(mass1, velocity1)
p2 = calculate_momentum(mass2, velocity2)
ke1 = calculate_kinetic_energy(mass1, velocity1)
ke2 = calculate_kinetic_energy(mass2, velocity2)

print(f"Ball 1: mass={mass1}, velocity={velocity1}")
print(f"  Momentum: {p1}")
print(f"  Kinetic Energy: {ke1:.2f}")
print(f"\nBall 2: mass={mass2}, velocity={velocity2}")
print(f"  Momentum: {p2}")
print(f"  Kinetic Energy: {ke2:.2f}")
print(f"\nTotal Momentum: {p1 + p2}")
print(f"Total Kinetic Energy: {ke1 + ke2:.2f}")

# %% [markdown]
# ## 2. Collision Detection
# 
# ### Circle-Circle Collision
# 
# Two circles collide when the distance between their centers is less than or equal to the sum of their radii:
# 
# $$d = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2} \leq r_1 + r_2$$

# %%
def check_collision(pos1, radius1, pos2, radius2):
    """Check if two circles are colliding"""
    distance = np.sqrt(np.sum((np.array(pos2) - np.array(pos1))**2))
    return distance <= (radius1 + radius2), distance

# Visualize collision detection
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Non-colliding circles
circle1 = Circle((2, 2), 1, color='red', alpha=0.7)
circle2 = Circle((6, 3), 1.5, color='blue', alpha=0.7)
ax1.add_patch(circle1)
ax1.add_patch(circle2)
ax1.plot([2, 6], [2, 3], 'white', linestyle='--', alpha=0.5)
ax1.set_xlim(0, 8)
ax1.set_ylim(0, 6)
ax1.set_aspect('equal')
ax1.set_title('Non-Colliding Circles')
ax1.grid(True, alpha=0.3)

# Colliding circles
circle3 = Circle((2, 2), 1, color='red', alpha=0.7)
circle4 = Circle((3.5, 2.5), 1.5, color='blue', alpha=0.7)
ax2.add_patch(circle3)
ax2.add_patch(circle4)
ax2.plot([2, 3.5], [2, 2.5], 'white', linestyle='--', alpha=0.5)
ax2.set_xlim(0, 6)
ax2.set_ylim(0, 5)
ax2.set_aspect('equal')
ax2.set_title('Colliding Circles')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Check the examples
collision1, dist1 = check_collision((2, 2), 1, (6, 3), 1.5)
collision2, dist2 = check_collision((2, 2), 1, (3.5, 2.5), 1.5)

print(f"Example 1: Distance = {dist1:.2f}, Sum of radii = {1 + 1.5}, Colliding = {collision1}")
print(f"Example 2: Distance = {dist2:.2f}, Sum of radii = {1 + 1.5}, Colliding = {collision2}")

# %% [markdown]
# ## 3. Collision Response Mathematics
# 
# ### The Collision Normal Vector
# 
# The collision normal vector $\hat{n}$ points from the center of the first ball to the center of the second ball:
# 
# $$\hat{n} = \frac{\vec{r_2} - \vec{r_1}}{|\vec{r_2} - \vec{r_1}|}$$
# 
# where $\vec{r_1}$ and $\vec{r_2}$ are the position vectors of the two balls.

# %%
def calculate_collision_normal(pos1, pos2):
    """Calculate the collision normal vector"""
    diff = np.array(pos2) - np.array(pos1)
    distance = np.linalg.norm(diff)
    if distance == 0:
        return np.array([1, 0])  # Arbitrary direction if positions are identical
    return diff / distance

# Example
pos1, pos2 = [2, 2], [5, 4]
normal = calculate_collision_normal(pos1, pos2)
print(f"Position 1: {pos1}")
print(f"Position 2: {pos2}")
print(f"Collision Normal: {normal}")
print(f"Normal magnitude: {np.linalg.norm(normal):.6f}")

# %% [markdown]
# ### Relative Velocity Along the Normal
# 
# The relative velocity along the collision normal determines if the objects are approaching or separating:
# 
# $$v_{rel} = (\vec{v_2} - \vec{v_1}) \cdot \hat{n}$$
# 
# - If $v_{rel} > 0$: Objects are separating (no collision response needed)
# - If $v_{rel} < 0$: Objects are approaching (collision response needed)

# %%
def relative_velocity_normal(vel1, vel2, normal):
    """Calculate relative velocity along the collision normal"""
    relative_velocity = np.array(vel2) - np.array(vel1)
    return np.dot(relative_velocity, normal)

# Example
vel1, vel2 = [10, 5], [-5, 8]
normal = [0.6, 0.8]  # Normalized collision normal
v_rel = relative_velocity_normal(vel1, vel2, normal)

print(f"Velocity 1: {vel1}")
print(f"Velocity 2: {vel2}")
print(f"Collision Normal: {normal}")
print(f"Relative Velocity Along Normal: {v_rel:.2f}")
print(f"Objects are {'separating' if v_rel > 0 else 'approaching'}")

# %% [markdown]
# ## 4. Impulse-Based Collision Response
# 
# ### The Impulse Formula
# 
# The impulse $J$ applied to each ball is calculated using:
# 
# $$J = \frac{-(1 + e) \cdot v_{rel}}{m_1^{-1} + m_2^{-1}}$$
# 
# where:
# - $e$ is the coefficient of restitution (1 for perfectly elastic collisions)
# - $v_{rel}$ is the relative velocity along the collision normal
# - $m_1, m_2$ are the masses of the two balls

# %%
def calculate_impulse(mass1, mass2, relative_velocity_normal, restitution=1.0):
    """Calculate collision impulse"""
    return -(1 + restitution) * relative_velocity_normal / (1/mass1 + 1/mass2)

# Example impulse calculation
mass1, mass2 = 5.0, 3.0
v_rel = -8.0  # Negative means approaching
impulse = calculate_impulse(mass1, mass2, v_rel)

print(f"Mass 1: {mass1}")
print(f"Mass 2: {mass2}")
print(f"Relative Velocity (normal): {v_rel}")
print(f"Impulse: {impulse:.2f}")

# %% [markdown]
# ### Velocity Updates
# 
# After calculating the impulse, the velocities are updated:
# 
# $$\vec{v_1'} = \vec{v_1} + \frac{J}{m_1} \hat{n}$$
# $$\vec{v_2'} = \vec{v_2} - \frac{J}{m_2} \hat{n}$$

# %%
def resolve_collision(pos1, pos2, vel1, vel2, mass1, mass2, restitution=1.0):
    """Complete collision resolution"""
    # Calculate collision normal
    normal = calculate_collision_normal(pos1, pos2)
    
    # Calculate relative velocity along normal
    v_rel = relative_velocity_normal(vel1, vel2, normal)
    
    # Don't resolve if objects are separating
    if v_rel > 0:
        return vel1, vel2
    
    # Calculate impulse
    impulse = calculate_impulse(mass1, mass2, v_rel, restitution)
    
    # Update velocities
    vel1_new = np.array(vel1) + (impulse / mass1) * normal
    vel2_new = np.array(vel2) - (impulse / mass2) * normal
    
    return vel1_new.tolist(), vel2_new.tolist()

# Example collision resolution
pos1, pos2 = [2, 2], [4, 3]
vel1, vel2 = [10, 0], [-5, 0]
mass1, mass2 = 5.0, 3.0

print("Before collision:")
print(f"  Ball 1: pos={pos1}, vel={vel1}, mass={mass1}")
print(f"  Ball 2: pos={pos2}, vel={vel2}, mass={mass2}")

# Calculate initial momentum and energy
p1_initial = calculate_momentum(mass1, vel1)
p2_initial = calculate_momentum(mass2, vel2)
ke1_initial = calculate_kinetic_energy(mass1, vel1)
ke2_initial = calculate_kinetic_energy(mass2, vel2)

print(f"  Total initial momentum: {p1_initial + p2_initial}")
print(f"  Total initial kinetic energy: {ke1_initial + ke2_initial:.2f}")

# Resolve collision
vel1_new, vel2_new = resolve_collision(pos1, pos2, vel1, vel2, mass1, mass2)

print(f"\nAfter collision:")
print(f"  Ball 1: pos={pos1}, vel={vel1_new}, mass={mass1}")
print(f"  Ball 2: pos={pos2}, vel={vel2_new}, mass={mass2}")

# Calculate final momentum and energy
p1_final = calculate_momentum(mass1, vel1_new)
p2_final = calculate_momentum(mass2, vel2_new)
ke1_final = calculate_kinetic_energy(mass1, vel1_new)
ke2_final = calculate_kinetic_energy(mass2, vel2_new)

print(f"  Total final momentum: {p1_final + p2_final}")
print(f"  Total final kinetic energy: {ke1_final + ke2_final:.2f}")

# Verify conservation
momentum_error = np.linalg.norm((p1_